In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [ ]:
class PlayingCardDataset(Dataset):
  def __init__(self, data_dir, transform=None):
    self.data= ImageFolder(data_dir, transform=transform)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

  @property
  def classes(self):
    return self.data.classes

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gpiosenka/cards-image-datasetclassification")

print("Path to dataset files:", path)

In [ ]:
dataset = PlayingCardDataset(data_dir='/kaggle/input/cards-image-datasetclassification/train')

In [ ]:
len(dataset)

In [ ]:
target_to_class = {v: k for k, v in ImageFolder('/kaggle/input/cards-image-datasetclassification/train').class_to_idx.items()}

In [ ]:
target_to_class

In [ ]:
# Transforming the dataset
transform= transforms.Compose([transforms.Resize((128,128)), transforms.ToTensor(),])

In [ ]:
data_dir = '/kaggle/input/cards-image-datasetclassification/train'
dataset = PlayingCardDataset(data_dir, transform)  #Uploading the transformed dataset

In [ ]:
# Printing a random card image and the corresponding class that it belongs to
image, label = dataset[100]
image


In [ ]:
image.shape  #Converted to vector

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
# To retreive the first batch of data and then loop over it
for images, labels in dataloader:
    break

In [ ]:
images.shape  #32 inputs in one batch

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2= nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2= nn.MaxPool2d(kernel_size=2, stride=2)


        self.fc1= nn.Linear(32*32*32, num_classes)
        self.dropout = nn.Dropout(0.5)



    def forward(self, x):
        x=self.pool1(self.relu1(self.conv1(x)))
        x=self.pool2(self.relu2(self.conv2(x)))
        x=x.view(x.size(0), -1)  # Flattening 3D features into 1D vector
        x=self.fc1(x)
        x= self.dropout(x)
        return x

In [ ]:
num_classes= len(dataset.classes)
print(num_classes)

model= CNN(num_classes= num_classes)

In [ ]:
cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.to("cpu")

In [ ]:
num_epochs = 5
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0

    for images, labels in tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images = images.to("cpu")
        labels = labels.to("cpu")

        optimizer.zero_grad()

        outputs = model(images)
        loss = cross_entropy_loss(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(dataloader)
    print(f"✅ Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.4f}")


In [ ]:
model.eval()  # Set model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in dataloader:
        images = images.to("cpu")
        labels = labels.to("cpu")

        outputs = model(images)

        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate accuracy
accuracy = 100 * correct / total
print(f"Accuracy on training data: {accuracy:.2f}%")


In [ ]:
# Load test set
test_data_dir = '/kaggle/input/cards-image-datasetclassification/test'
test_dataset = PlayingCardDataset(test_data_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32)


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to("cpu"), labels.to("cpu")
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = correct / total
print(f" Test Accuracy: {test_accuracy:.4f}")
